In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load data
file_path = '/content/Dataset-3.xlsx'
sheet1_df = pd.read_excel(file_path, sheet_name='30-10-DISEASE VS BODYFN')
sheet2_df = pd.read_excel(file_path, sheet_name='31-10 BODYFN vs assistive techn')
sheet3_df = pd.read_excel(file_path, sheet_name='31-10 BODYFN vs HOME ICF')

# Preprocess: Lowercase and strip whitespace for consistent matching
for df in [sheet1_df, sheet2_df, sheet3_df]:
    df['Symptoms'] = df['Symptoms'].str.lower().str.strip()
sheet1_df['Disease'] = sheet1_df['Disease'].str.lower().str.strip()

# Combine symptoms from all sheets to fit the TF-IDF vectorizer
all_symptoms = pd.concat([sheet1_df['Symptoms'], sheet2_df['Symptoms'], sheet3_df['Symptoms']]).unique()

# Create TF-IDF matrix for all symptoms
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_symptoms)

# Convert the TF-IDF matrix back into individual DataFrames for each sheet
sheet1_vectors = vectorizer.transform(sheet1_df['Symptoms'])
sheet2_vectors = vectorizer.transform(sheet2_df['Symptoms'])
sheet3_vectors = vectorizer.transform(sheet3_df['Symptoms'])

# Function to get symptoms, assistive technology, and home functions for a disease
def get_disease_info(disease_name, similarity_threshold=0.5):
    disease_name = disease_name.lower().strip()

    # Step 1: Find symptoms related to the disease in sheet 1
    symptoms = sheet1_df[sheet1_df['Disease'] == disease_name]['Symptoms'].tolist()

    if not symptoms:
        return f"No data found for disease: {disease_name}"

    assistive_tech_matches = []
    home_function_matches = []

    for symptom in symptoms:
        # Vectorize the symptom
        symptom_vector = vectorizer.transform([symptom])

        # Step 2: Compute similarity with all symptoms in sheet2 and sheet3
        similarity_to_sheet2 = cosine_similarity(symptom_vector, sheet2_vectors).flatten()
        similarity_to_sheet3 = cosine_similarity(symptom_vector, sheet3_vectors).flatten()

        # Find the most similar symptom in sheet2 (Assistive Technology)
        max_sim_idx_2 = similarity_to_sheet2.argmax()
        if similarity_to_sheet2[max_sim_idx_2] >= similarity_threshold:
            assistive_tech_matches.append({
                "Symptom": sheet2_df['Symptoms'].iloc[max_sim_idx_2],
                "Assistive Technology": sheet2_df['Assistive Technology'].iloc[max_sim_idx_2]
            })

        # Find the most similar symptom in sheet3 (Home Functions)
        max_sim_idx_3 = similarity_to_sheet3.argmax()
        if similarity_to_sheet3[max_sim_idx_3] >= similarity_threshold:
            home_function_matches.append({
                "Symptom": sheet3_df['Symptoms'].iloc[max_sim_idx_3],
                "Home Function": sheet3_df['Home functions'].iloc[max_sim_idx_3]
            })

    # Display results
    print(f"Disease: {disease_name.capitalize()}\n")

    print("Symptoms:")
    for symptom in symptoms:
        print(f"  - {symptom.capitalize()}")

    print("\nAssistive Technology:")
    if assistive_tech_matches:
        for match in assistive_tech_matches:
            print(f"  - Symptom: {match['Symptom'].capitalize()}, Assistive Technology: {match['Assistive Technology']}")
    else:
        print("  No matching assistive technology found.")

    print("\nHome Functions:")
    if home_function_matches:
        for match in home_function_matches:
            print(f"  - Symptom: {match['Symptom'].capitalize()}, Home Function: {match['Home Function']}")
    else:
        print("  No matching home function found.")

# Example usage: Input the disease name
disease_name = input("Enter the disease name: ")
get_disease_info(disease_name)


Enter the disease name: Asthma
Disease: Asthma

Symptoms:
  - Shortness of breath, wheezing, coughing, chest tightness

Assistive Technology:
  - Symptom: Shortness of breath, Assistive Technology: Oxygen therapy devices

Home Functions:
  - Symptom: Shortness of breath, Home Function: Open spaces for better airflow


In [ ]:
!pip install tabulate


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate  # Import tabulate for bordered tables

# Load data
file_path = '/content/Dataset-3.xlsx'
sheet1_df = pd.read_excel(file_path, sheet_name='30-10-DISEASE VS BODYFN')
sheet2_df = pd.read_excel(file_path, sheet_name='31-10 BODYFN vs assistive techn')
sheet3_df = pd.read_excel(file_path, sheet_name='31-10 BODYFN vs HOME ICF')

# Preprocess: Lowercase and strip whitespace for consistent matching
for df in [sheet1_df, sheet2_df, sheet3_df]:
    df['Symptoms'] = df['Symptoms'].str.lower().str.strip()
sheet1_df['Disease'] = sheet1_df['Disease'].str.lower().str.strip()

# Combine all symptoms for TF-IDF fitting
all_symptoms = pd.concat([sheet1_df['Symptoms'], sheet2_df['Symptoms'], sheet3_df['Symptoms']]).unique()

# Create a TF-IDF vectorizer and fit it on all symptoms
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_symptoms)

# Map symptoms to their TF-IDF vectors for each sheet
sheet1_vectors = vectorizer.transform(sheet1_df['Symptoms'])
sheet2_vectors = vectorizer.transform(sheet2_df['Symptoms'])
sheet3_vectors = vectorizer.transform(sheet3_df['Symptoms'])

# Function to get symptoms, assistive technology, and home functions for a disease
def get_disease_info(disease_name, similarity_threshold=0.3):
    disease_name = disease_name.lower().strip()

    # Step 1: Find symptoms related to the disease in sheet 1
    symptoms = sheet1_df[sheet1_df['Disease'] == disease_name]['Symptoms'].tolist()

    if not symptoms:
        return f"No data found for disease: {disease_name}"

    assistive_tech_matches = []
    home_function_matches = []

    for symptom in symptoms:
        # Vectorize the symptom
        symptom_vector = vectorizer.transform([symptom])

        # Step 2: Compute similarity with all symptoms in sheet2 and sheet3
        similarity_to_sheet2 = cosine_similarity(symptom_vector, sheet2_vectors).flatten()
        similarity_to_sheet3 = cosine_similarity(symptom_vector, sheet3_vectors).flatten()

        # Find relevant matches in sheet2 (Assistive Technology)
        for i, sim_score in enumerate(similarity_to_sheet2):
            if sim_score >= similarity_threshold:
                assistive_tech_matches.append({
                    "Symptom": sheet2_df['Symptoms'].iloc[i],
                    "Assistive Technology": sheet2_df['Assistive Technology'].iloc[i]
                })

        # Find relevant matches in sheet3 (Home Functions)
        for i, sim_score in enumerate(similarity_to_sheet3):
            if sim_score >= similarity_threshold:
                home_function_matches.append({
                    "Symptom": sheet3_df['Symptoms'].iloc[i],
                    "Home Function": sheet3_df['Home functions'].iloc[i]
                })

    # Display results in tabular format with borders
    print(f"Disease: {disease_name.capitalize()}\n")

    # Symptoms Table
    symptoms_df = pd.DataFrame({"Symptoms": symptoms})
    print("Symptoms:")
    print(tabulate(symptoms_df, headers="keys", tablefmt="grid", showindex=False))

    # Assistive Technology Table
    if assistive_tech_matches:
        assistive_tech_df = pd.DataFrame(assistive_tech_matches)
        print("\nRelevant Assistive Technology:")
        print(tabulate(assistive_tech_df, headers="keys", tablefmt="grid", showindex=False))
    else:
        print("\nRelevant Assistive Technology:\n  No matching assistive technology found.")

    # Home Functions Table
    if home_function_matches:
        home_functions_df = pd.DataFrame(home_function_matches)
        print("\nRelevant Home Functions:")
        print(tabulate(home_functions_df, headers="keys", tablefmt="grid", showindex=False))
    else:
        print("\nRelevant Home Functions:\n  No matching home functions found.")

# Example usage: Input the disease name
disease_name = input("Enter the disease name: ")
get_disease_info(disease_name)


Enter the disease name: Asthma
Disease: Asthma

Symptoms:
+----------------------------------------------------------+
| Symptoms                                                 |
+==========================================================+
| shortness of breath, wheezing, coughing, chest tightness |
+----------------------------------------------------------+

Relevant Assistive Technology:
+---------------------+--------------------------+
| Symptom             | Assistive Technology     |
+=====================+==========================+
| chest pain          | Heart monitoring devices |
+---------------------+--------------------------+
| coughing            | Humidifiers              |
+---------------------+--------------------------+
| shortness of breath | Oxygen therapy devices   |
+---------------------+--------------------------+
| wheezing            | Breathing aids           |
+---------------------+--------------------------+

Relevant Home Functions:
+-----------------